<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Mixed_Integer_Problem_for_Draft_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Initialisierung und Konfiguration

## 1.1 Bibliotheken installieren und laden


In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


## 1.2 Liga-Parameter und Rahmenbedingungen definieren

In [2]:
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# 2 Datenbeschaffung und -vorverarbeitung


## 2.1 ADP, Bye Weeks und Projections laden

In [3]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)


bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)


season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
      .drop_duplicates(subset='Player', keep='first')
)
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)
week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,
        season_projections['avg_proj']
    )
season_projections = season_projections.drop(columns='avg_proj')

## 2.2 Berechnung von TTL, Dropoff und VOR

In [4]:
dropoff_w = {'QB':1.0,'RB':1.0,'WR':1.0,'TE':0.9,'K':0.4,'DST':0.3}
vor_w     = {'QB':0.8,'RB':1.0,'WR':1.0,'TE':0.8,'K':0.25,'DST':0.25}

rep_ttl = (
    season_projections
      .groupby('POS')['TTL']
      .apply(lambda s: s.nlargest(lineup_req[s.name]*num_teams)
                   .iloc[-1]
             if len(s) >= lineup_req[s.name]*num_teams else 0)
)

season_projections = (
    season_projections
      .sort_values(['POS','TTL'], ascending=[True,False])
      .assign(
          dropoff=lambda df: (
              df.groupby('POS')['TTL']
                .diff(-1)
                .fillna(0)
                .mul(df['POS'].map(dropoff_w))
          ),
          VOR=lambda df: (
              (df['TTL'] - df['POS'].map(rep_ttl))
                .clip(lower=0)
                .mul(df['POS'].map(vor_w))
          )
      )
      .fillna({'dropoff': 0, 'VOR': 0, 'ADP': 999})
      .sort_values(['ADP', 'TTL'], ascending=[True, False])
      .reset_index(drop=True)
)


# 3 Modellparameter und Variablenaufbau

## 3.1 Spielerlisten und Parameter-Mapping

In [16]:
# 1) Spieler-DataFrame mit Rang
players = (
    season_projections[['Player','POS','ADP','TTL']]
    .fillna({'ADP': 999})
    .sort_values(['ADP','TTL'], ascending=[True, False])
    .assign(Rank=lambda df: df.reset_index(drop=True).index + 1)
    .reset_index(drop=True)
)
players_list = players['Player'].tolist()
Rk = dict(zip(players_list, players['Rank']))

# 2) Basis-Parameter
positions      = {'QB','RB','WR','TE','K','DST','FLEX'}
flex_eligible  = {'RB','WR','TE'}
weeks          = list(range(1,18))
pos_map        = season_projections.set_index('Player')['POS'].to_dict()
lineup_req     = {'QB':1,'RB':2,'WR':2,'TE':1,'K':1,'DST':1,'FLEX':1}
max_req        = {'QB':3,'RB':999,'WR':999,'TE':2,'K':1,'DST':1}
week_cols      = [c for c in season_projections.columns if c.startswith('Week_')]

# 3) Projektionen-Dictionary
season_projections_dict = {
    player: {
        **{int(c.split('_')[1]): val for c,val in row[week_cols].items()},
        'dropoff': row['dropoff'],
        'VOR':     row['VOR']
    }
    for player,row in season_projections.set_index('Player').iterrows()
}

# 3.2) VOR-Dictionary
vor_dict = {p: season_projections_dict[p]['VOR'] for p in season_projections_dict}

# 4) Weitere Konstanten
beta              = dict.fromkeys(weeks, 120.0)
weight_score      = 1.0    # entspricht ehem. lambda_0
weight_win_rs     = 3      # entspricht ehem. lambda_1
weight_win_po     = 4      # entspricht ehem. lambda_2
weight_dropoff    = 0.5    # entspricht ehem. lambda_3
weight_vor        = 4      # entspricht ehem. lambda_4
df_sorted         = players.copy()
topk_pct          = 0.0025

# 5) Teams & Draft-Reihenfolge
teams       = [f'Team {i+1}' for i in range(num_teams)]
DM_team     = teams[0]
draft_order = [
    team
    for rnd in range(num_rounds)
    for team in (teams if rnd % 2 == 0 else teams[::-1])
]

# 6) Gegner-Pick (kompakt)
def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
    rem   = sorted(available, key=Rk.get)
    topk  = max(1, int(len(rem)*topk_pct))
    deficits = {
        pos: lineup_req[pos] - sum(pos_map[p]==pos for p in roster)
        for pos in lineup_req if pos!='FLEX'
    }
    needed = [pos for pos,d in deficits.items() if d>0]
    pool   = [p for p in rem if pos_map[p] in needed] or rem[:topk]
    return random.choice(pool[:topk])

# 7) Draft-Initialisierung
rosters     = {tm: [] for tm in teams}
available   = set(players_list)
draft_log   = []


## 3.2 Optimierungsvariablen und Zielfunktion konfigurieren

In [13]:
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

def optimize_pick(
    team,
    pick_idx,
    picks_remaining,
    rosters,
    remaining_players,
    f_dict,
    vor_dict,
    pos_map,
    lineup_req,
    max_req,
    positions,
    flex_eligible,
    weeks,
    beta,
    alpha,
    teams,
    num_teams,
    Rk
):
    # 1) Modell initialisieren
    m = Model(sense=maximize, solver_name='CBC')

    # 2) Decision-Vars
    y = {i: m.add_var(var_type=BINARY, name=f'y_{i}')               for i in remaining_players}
    x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}')  for i in remaining_players for t in weeks}
    z = {t: m.add_var(var_type=BINARY, name=f'z_{t}')               for t in weeks}

    # 3) Zielfunktion sauber ausgeschrieben
    m.objective = (
        weight_score     * xsum(f_dict[i][t] * x[i, t]        for i in remaining_players for t in weeks)
      + weight_win_rs    * xsum(z[t]                          for t in weeks[:15])
      + weight_win_po    * xsum(z[t]                          for t in weeks[15:])
      + weight_dropoff   * xsum(f_dict[i]['dropoff'] * y[i]   for i in remaining_players)
      - weight_vor       * xsum(vor_dict.get(i, 0.0) * y[i]   for i in remaining_players)
    )

    # 4) Bereits gedraftete Spieler fixieren
    for p in rosters[team]:
        m += y[p] == 1

    # 5) Anzahl neuer Picks
    m += xsum(y[i] for i in remaining_players) == picks_remaining

    # 6) Mindest-Positionen (ohne FLEX)
    for pos_name, req in lineup_req.items():
        if pos_name != 'FLEX':
            have = sum(1 for p in rosters[team] if pos_map[p] == pos_name)
            need = max(0, req - have)
            m += xsum(y[i] for i in remaining_players
                      if pos_map[i] == pos_name) >= need

    # 7) Obergrenzen & Wochen-Constraints
    for t in weeks:
        # Obergrenzen pro Position
        for j in positions - {'FLEX'}:
            m += xsum(y[i] for i in remaining_players
                      if pos_map[i] == j) <= max_req[j]

        # Jeder Spieler max. einen Slot pro Woche
        for i in remaining_players:
            star = x[i, t] if pos_map[i] in positions - {'FLEX'} else 0
            flex = x[i, t] if pos_map[i] in flex_eligible   else 0
            m += star + flex <= 1
            m += x[i, t] <= y[i]

        # Win-Variable
        m += z[t] <= xsum(f_dict[i][t] * x[i, t]
                          for i in remaining_players) / beta[t]

    # 8) Vorausschau-Constraint (Top-alpha)
    n_k = pick_idx
    for future_pick in range(pick_idx + 1,
                             pick_idx + picks_remaining * len(teams),
                             len(teams)):
        top_cut = int(alpha * (future_pick - n_k))
        if top_cut > 0:
            top_players = [i for i, r in Rk.items() if r <= top_cut]
            m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

    # 9) Lösen
    m.optimize()
    if m.num_solutions == 0:
        raise RuntimeError(f'No feasible solution at pick {pick_idx}')

    # 10) Gewählten Spieler extrahieren
    picks = [i for i in remaining_players
             if y[i].x >= 0.99 and i not in rosters[team]]
    return min(picks, key=lambda i: Rk[i])


# 3.3 Ausspielen des Drafts

In [17]:
# 8) Draft-Schleife: DM- und Gegner-Picks durchführen
for pick_idx, team in enumerate(draft_order, start=1):
    # verbleibende Picks und Ranking neu berechnen
    picks_remaining   = num_rounds - len(rosters[team])
    remaining_players = sorted(available, key=lambda p: Rk[p])
    Rk                = {p: i+1 for i, p in enumerate(remaining_players)}

    if team == DM_team:
        pick = optimize_pick(
            team,
            pick_idx,
            picks_remaining,
            rosters,
            remaining_players,
            season_projections_dict,   # f_dict
            vor_dict,
            pos_map,
            lineup_req,
            max_req,
            positions,
            flex_eligible,
            weeks,
            beta,
            alpha,
            teams,
            num_teams,
            Rk
        )
    else:
        pick = opponent_pick(rosters[team], available, Rk, lineup_req)

    # Pick übernehmen
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Team': team,
        'Pick': pick_idx,
        'Player': pick,
        'POS': pos_map[pick]
    })


KeyError: 'Breece Hall'

# 4 Entwurfsphase (Draft-Optimierung)

## 4.1 Draft-Reihenfolge generieren

## 4.2 Optimierungsmodell für den MIP-Draft

## 4.3 Gegnerische Picks simulieren

# 5 Ergebnisaufbereitung

## 5.1 Draft-Log exportieren und zusammenführen

## 5.2 Ergebnis-Statustext ausgeben

# 6 Evaluation